In [1]:
"""
import packages
"""
import os, glob
import pandas as pd
import numpy as np
import math, random
from datetime import datetime
import pickle
from matplotlib import pyplot as plt
import argparse
import path, sys, re, time
from collections import Counter
from scipy.spatial import distance_matrix
from scipy.signal import find_peaks

In [2]:
"""
import custom modules
"""
from module_.info.config import config
from module_.text2array import text2array_adlmr as ttaa
from module_.win2feat import sliding_window
from module_.cpd import change_point_detection as cpd
from module_.evaluation import evaluation_adlmr as ea
from module_.analysis import neighbor_events as ne

# adlmr (CASAS, 2-resident, episodes)

In [3]:
# """
# load data by episode: mixed, group, single
# """
# raw_episodes, single_episodes={}, {}

# with open("dataset/adlmr/annotated", 'rb') as f: 
#     text=f.readlines()
#     _, group_stream, _, _, _=ttaa(text)
#     group_episodes={idx:item for idx, item in enumerate(group_stream)}

# for i in range(1, 27):
#     name=str(i) if len(str(i))!=1 else '0'+str(i)
#     with open("dataset/adlmr/P{}.txt".format(name), 'rb') as f: 
#         text=f.readlines()
#     single_stream, _, raw_stream, t_single, t_raw=ttaa(text) # NEXT: set transition points
#     raw_episodes['P'+name]={'stream':raw_stream, 'transition':t_raw}
#     single_episodes['P'+name]={'stream':single_stream, 'transition': t_single}

In [58]:
# with open("dataset/adlmr/annotated", 'rb') as f:
#     text=f.readlines()
#     _, _, raw_stream, _, t_raw=ttaa(text)

# sensor_list=sorted(set(np.array(raw_stream)[:,0]))
# windows=sliding_window(np.array(raw_stream))

"""
load single stream
"""

dir_="preprocessed/adlmr/group/data.pickle"
with open(dir_,'rb') as f:
    data=pickle.load(f)

"""
raw stream
"""

# total=[]
# total_atd=[]

# for _, df in data.items():
#     total+=df['stream']

# activity={'1':'1', '2':'2'}
# for idx, event in enumerate(total):
#     if event[3]!='0':
#         if activity[event[3]]!=event[4]: # different activity
#             total_atd.append(idx)
#             activity[event[3]]=event[4]
    
#     if event[5]!='0':
#         if activity[event[5]]!=event[6]: # different activity
#             total_atd.append(idx)
#             activity[event[5]]=event[6]
# len(total_atd)

"""
group stream
"""

group=[]
group_atd=[]
for _, df in data.items():
    group_atd.append(len(group)+len(df))
    group+=df
group_atd=group_atd[:-1]
print(len(group), len(group_atd))
"""
single stream
"""

# one, two=[], []
# for _, df in data.items():
#     one+=df['stream']['1']
#     two+=df['stream']['2']

# print(len(one), len(two))

# one_atd, two_atd=[],[]
# activity={'1':'1', '2':'2'}
# for idx, event in enumerate(one):
#     if event[3]!='0':
#         if event[3]=='1':
#             if activity['1']!=event[4]:
#                 one_atd.append(idx)
#                 activity['1']=event[4]
#     if event[5]!='0':
#         if event[5]=='1':
#             if activity['1']!=event[6]:
#                 one_atd.append(idx)
#                 activity['1']=event[6]

# for idx, event in enumerate(two):
#     if event[3]!='0':
#         if event[3]=='2':
#             if activity['2']!=event[4]:
#                 two_atd.append(idx)
#                 activity['2']=event[4]
#     if event[5]!='0':
#         if event[5]=='2':
#             if activity['2']!=event[6]:
#                 two_atd.append(idx)
#                 activity['2']=event[6]

# print(len(one_atd), len(two_atd))

"""
CPD
"""

# sensor_list=sorted(set(np.array(group)[:,0]))
# windows=sliding_window(np.array(group))
# features, scores, params, changes=cpd(windows, sensor_list, data_name='adlmr', message='group')

# message='group'
# np.save("scores/adlmr/f_{}.npy".format(message), features)
# np.save("scores/adlmr/s_{}.npy".format(message), scores)
# np.save("scores/adlmr/p_{}.npy".format(message), params)

"""

"""

# for key in episodes.keys():
#     print(key)
#     eps=np.array(episodes[key])
#     # trs_=np.array(trs_dict[key])

#     sensor_set=set(eps[:,0])
#     windows=sliding_window(eps)
#     features, scores, params=cpd(windows, sensor_set, data_name='adlmr')

#     dirname="scores/adlmr/{}".format(key)
#     if not os.path.isdir(dirname):
#         os.mkdir(dirname)
#     np.save("{}/f.npy".format(dirname), features)
#     np.save("{}/s.npy".format(dirname), scores)
#     np.save("{}/p.npy".format(dirname), params)

#     print("Done")

9042 126


'\n\n'

In [66]:
target='group'
scores=np.load("scores/adlmr/s_{}.npy".format(target))
features=np.load("scores/adlmr/f_{}.npy".format(target))
params=np.load("scores/adlmr/p_{}.npy".format(target))

"""
candidates
"""
events_=group
eval_transitions=group_atd

threshold=0.3
# candidates_all=[idx if eval_scores[idx]>threshold else 0 for idx in range(cut)]
candidates_all=[idx for idx in range(len(scores)) if scores[idx]>threshold]
peaks, _=find_peaks(scores)
# candidates_peak=[idx if idx in peaks and eval_scores[idx]>threshold else 0 for idx in range(cut)]
candidates_peak=[idx for idx in range(len(scores)) if idx in peaks and scores[idx]>threshold]
len(eval_transitions), len(candidates_all), len(candidates_peak)

"""
measurement
"""
transitions_time=np.array([float(events_[idx][2]) for idx in eval_transitions])

tp, tn, fp, fn=[],[],[],[]

cand=candidates_peak

for i in range(len(scores)):
    if i in cand: # positive
        if i in eval_transitions:
            tp.append(i)
            continue
        timestamp_=float(events_[i][2])
        if sum(abs(transitions_time-timestamp_)<10)!=0:
            tp.append(i)
            continue
        fp.append(i)
    else:
        if i in eval_transitions:
            fn.append(i)
            continue
        tn.append(i)
tp_, tn_, fp_, fn_=len(tp), len(tn), len(fp), len(fn)
print(tp_, tn_, fp_, fn_)
print(tp_/(tp_+fn_))
print(fp_/(fp_+tn_))

293 6906 1725 118
0.7128953771289538
0.19986096628432395


In [57]:
np.array(group_atd)

array([  71,  136,  153,  305,  404,  445,  491,  540,  688,  785,  847,
        901, 1006, 1181, 1252, 1314, 1344, 1425, 1596, 1670, 1705, 1768,
       1851, 2083, 2156, 2206, 2230, 2269, 2390, 2449, 2501, 2541, 2599,
       2672, 2740, 2783, 2821, 2900, 3051, 3135, 3195, 3256, 3299, 3344,
       3386, 3478, 3648, 3699, 3747, 3792, 3823, 3903, 3965, 4031, 4049,
       4108, 4207, 4288, 4317, 4339, 4428, 4496, 4512, 4557, 4586, 4640,
       4773, 4845, 4932, 4965, 5016, 5080, 5155, 5189, 5202, 5236, 5375,
       5416, 5441, 5486, 5563, 5678, 5754, 5802, 5825, 5869, 6071, 6165,
       6198, 6226, 6290, 6440, 6531, 6563, 6621, 6658, 6769, 6834, 6855,
       6916, 6990, 7057, 7104, 7191, 7240, 7418, 7477, 7518, 7584, 7635,
       7753, 7827, 7882, 7922, 7963, 8096, 8157, 8216, 8238, 8288, 8470,
       8554, 8620, 8653, 8744, 8946, 9042])

In [149]:
from module_.helper.labeling import feature_label as fl
from module_.info.config import feature_name as fn

sensor_list=sorted(set(np.array(events_)[:,0]))
# print(features.shape)
# idx=candidates_peak[3]
# idx=group_atd[32]; print(idx)
idx=2584; print(idx)

# print(idx, events_[idx])
for i in range(32):
    idx_=max(0, idx-30+i)
    print(idx-30+i, events_[idx_])
views=np.array(features[idx])
# cols=[]
cols_=[]
for cnum in range(views.shape[1]):
    col=views[:,cnum]
    if max(col)!=min(col):
        # cols.append(cnum)
        cols_.append(np.array(col).reshape((-1,1)))
# print(cols, len(cols))
feature=np.concatenate(cols_, axis=1)
# print(views)
cf=fl(np.array(views), sensor_list, fn)

fd={}
for i, v in enumerate(cf.values()):
    # print(v)
    f=feature[:,i]
    fd[v]=np.round(abs(f[0]-f[-1]), 3)
print(scores[idx])
sort_fd={k: v for k, v in sorted(fd.items(), key=lambda item: item[1], reverse=True)}
print(sort_fd)
print(sum(sort_fd.values()))


2584
2554 ['M13', 'ON', 1226879088.529429, '2', '11', '0', '0']
2555 ['M13', 'OFF', 1226879090.09085, '2', '11', '0', '0']
2556 ['M13', 'ON', 1226879099.778959, '2', '11', '0', '0']
2557 ['M14', 'ON', 1226879100.085429, '2', '11', '0', '0']
2558 ['M14', 'OFF', 1226879101.502269, '2', '11', '0', '0']
2559 ['M13', 'OFF', 1226879101.827889, '2', '11', '0', '0']
2560 ['M13', 'ON', 1226879102.72088, '2', '11', '0', '0']
2561 ['M14', 'ON', 1226879110.68909, '2', '11', '0', '0']
2562 ['D07', 'CLOSE', 1226879120.50053, '1', '10', '0', '0']
2563 ['M14', 'OFF', 1226879121.580329, '2', '11', '0', '0']
2564 ['M13', 'OFF', 1226879123.74757, '2', '11', '0', '0']
2565 ['M13', 'ON', 1226879124.949419, '2', '11', '0', '0']
2566 ['M14', 'ON', 1226879125.523689, '2', '11', '0', '0']
2567 ['M13', 'OFF', 1226879127.96898, '2', '11', '0', '0']
2568 ['M14', 'OFF', 1226879129.21844, '2', '11', '0', '0']
2569 ['M14', 'ON', 1226879130.75899, '2', '11', '0', '0']
2570 ['M14', 'OFF', 1226879132.34024, '2', '11', 

In [5]:
# for key in episodes.keys():
#     eps_dir="scores/adlmr"
#     eps=np.array(episodes[key])
#     f_=np.load("{}/{}/f.npy".format(eps_dir, key))
#     s_=np.load("{}/{}/s.npy".format(eps_dir, key))
#     p_=np.load("{}/{}/p.npy".format(eps_dir, key))
#     t_=np.load("{}/{}/t.npy".format(eps_dir, key))

#     fp, fn, tp, tn = ea(scores, eps, t_)
#     tpr, fpr=len(tp)/(len(tp)+len(fn)), len(fp)/(len(fp)+len(tn)) 
#     print("[{}] FP: {} FN: {} TP: {} TN: {}, TPR: {}, FPR: {}".format(key, len(fp), len(fn), len(tp), len(tn), tpr, fpr))
#     print(t_, len(t_))

#     # target_=fp
#     target_=fp
#     f__=f_[np.array(target_)]; s__=s_[np.array(target_)]; p__=p_[np.array(target_)]
#     v__=[(item[:2], item[2:]) for item in f__]
#     dist_x__=np.array([np.square(distance_matrix(item[0], item[0])) for _, item in enumerate(v__)])
#     dist_y__=np.array([np.square(distance_matrix(item[0], item[1])) for _, item in enumerate(v__)])
#     phi_x__=np.array([np.exp(-0.5*np.square(distance_matrix(item[0], item[0])) / (p__[k,0]**2)) for k, item in enumerate(v__)])
#     phi_y__=np.array([np.exp(-0.5*np.square(distance_matrix(item[0], item[1])) / (p__[k,0]**2)) for k, item in enumerate(v__)])
#     weight__=np.array([np.mean(item, axis=0)/p__[k,1] for k, item in enumerate(phi_y__)])
#     scores__=np.array([max(0, 1-np.dot(phi_x__[k], item).sum(axis=0)/2) for k, item in enumerate(weight__)])
    
#     j=40
    
#     print('{}/{}'.format(j, len(target_)))
#     print('target_\t', target_[j])
#     event_seg=ne(eps, target_[j])
#     sensor_=set(np.array(event_seg)[:,0])
#     print(sensor_, len(sensor_))
    
#     for k, item in enumerate(event_seg):
#         print(max(0, target_[j]-30+k),"\t", item, '*' if k==30 else "")

#     for k, item in enumerate(f__[j]):
#         print(np.round(item,2),'t', k-1)
#     print('(sigma, lambda_)', p__[j], scores__[j])
#     print(dist_x__[j], 'squared distance (x and x)')
#     print(dist_y__[j], 'squared distance (x and y)')
#     print('*'*30)
#     print(np.round(phi_x__[j],2))
#     print(np.round(phi_y__[j],2))
#     print(np.round(scores__[j],2))
#     print('-'*30)

#     break

NameError: name 'episodes' is not defined